# 6.3 Elasto-Plasticity


This tutorial considers Hencky-type [1] plasticity with isotropic hardening in 2d (plane strain).
It showcases the classes `IntegrationRuleSpace`, `NewtonCF` and `MinimizationCF`.

We begin with a classical formulation with an explcit yield surface that is probably more familiar to engineers. 
In a second stage, we reformulate the nonlinear evolution problems to a minimization problem which leads to a more streamlined implementation.

The essential model ingredients are given in terms of the dissipation potential $\Phi$ and the stress response 
(material law; plane-strain) from which one can build the stored energy density contribution $\Psi$, see eg, [2]. For convenience, we first introduce the strain energy density $\Psi^\text{e}$

\begin{align*}
\Psi^\text{e}(\epsilon) = \| \epsilon\|^2_{M} = \frac{E}{(1+\nu)(1-2\nu)}\left((1-2\nu)\epsilon:\epsilon + \nu \,\text{tr}(\epsilon)^2\right) = \epsilon : \mathbf{C} : \epsilon.
\end{align*}

Moreover, we consider the full strain $\epsilon$ to be the sum of the elastic $\epsilon^e$ and the plastic strain contribution $p$.
Conversely, the elastic strain is the given as 
\begin{equation} \epsilon^\text{e} = \epsilon - p .\end{equation}
The isotropic hardening variable will be denoted by $\alpha$ throught this tutorial. The hardening potential
density is simply chosen as
$\Psi^\text{h} = \frac{H\,\alpha^2}{2}$ such that the total energy density reads
\begin{equation*}
\Psi((\epsilon - p), \alpha) = (\epsilon - p) : \mathbf{C} : (\epsilon - p) + \frac{H}{2} \alpha^2.
\end{equation*}


## Formulation with explicit yield surface

Based on the principle of maximum dissipation we can abstractly formulate the dissipation potential as

\begin{align*}
\Phi(\dot{p}, \dot{\alpha}) = 
  \sup_{\sigma} \sup_{\beta} \inf_{\lambda \geq 0} 
    \: \sigma : \dot{p} + \beta \dot{\alpha} - \lambda (\left|\mathrm{dev} \sigma\right| - \sigma_Y(1 + \beta))
\end{align*}

where $\left|\mathrm{dev} \sigma\right| - \sigma_Y(1 + \beta)$ is the yield function, which represents the elastic
limit. The stationary conditions of the above functional read as

\begin{align*} 
  &\dot{p} = \lambda \frac{\partial \left|\mathrm{dev} \sigma\right|}{\partial \sigma} \\
  &\dot{\alpha} = \lambda -\sigma_Y \\
  &\{\lambda >= 0 \: \land \: (\left|\mathrm{dev} \sigma\right| - \sigma_Y(1 + \beta)) <= 0 \: \land \:
  \lambda(\left|\mathrm{dev} \sigma\right| - \sigma_Y(1 + \beta)) = 0\}
\end{align*}

which are regarded as evolution equations for $p$ and $\alpha$. The actual values of $\sigma$ and $\beta$ are
obtained through the constitutive relations provided by the framework of generalized standard materials, we have

\begin{align*}
&\frac{\partial\Psi((\epsilon - p), \alpha)}{\partial p} + \frac{\partial\Phi}{\partial \dot{p}} = 0 \quad \Rightarrow \quad \sigma = \mathbf{C} : (\epsilon - p) \\
&\frac{\partial\Psi((\epsilon - p), \alpha)}{\partial \alpha} + \frac{\partial\Phi}{\partial \dot{\alpha}} = 0
\quad \Rightarrow \quad \beta = - H \alpha .
\end{align*}




## Constrained minimization formulation


A semi-implicit time-discrete formulation of the model including hardening reads [3]:  
For the given state $(u^k,p^k, \alpha^k)$ find a new solution $(u^{k+1},p^{k+1})$ such that
\begin{align*}
\text{1.} & \: && \int\Psi(\epsilon(u^{k+1}), p^{k+1},p^{k}, \alpha^{k})+\Phi(p^{k+1},p^k)- f^{k+1}\cdot 
u^{k+1}\,dx \rightarrow \min \\
\text{2.} & \: && \alpha^{k+1}=\alpha^k+\sigma_y H \|p^{k+1}-p^k\| ,
\end{align*}
with
\begin{align*}
&\Psi(\epsilon(u), p, p^k, \alpha^k) = \frac{1}{2} \|\epsilon(u)-p\|_{M}^2 
    + \frac{1}{2} \left(\alpha^k + \sigma_y H \|p-p^k\|_{\varepsilon}\right)^2 \quad\text{and}\quad
\\
&\Phi(p,p^k) = \sigma_Y \|p-p^k\|_{\varepsilon},
\end{align*}

where $\Psi$ denotes the augmented stored energy density and $\|\bullet \|_{\varepsilon}$ indicates a perturbed norm in order to avoid divisions by zero in the evaluation of the time-discrete evolution equation derived from an incremental variational principle.

The state $u$ is spatially discretized by continuous Lagrange elements, the internal states $p$ and $\alpha$ reside at quadrature points, for which NGSolve has the notion of an integration rule space. Thus, the problem above can be decomposed into a "global" problem for the displacements and local problems for the interal variables. The latter can be solved individually for each quadrature point. However, there is a coupling between the local problems and
displacements, which has to be accounted for by what is called "algorithmically consistent linearization".


**References**
 1. [W. Han and B.D. Reddy: Mathematical Theory and Numerical Analysis, Springer 1999](https://www.springer.com/gp/book/9781461459392)
 2. [K. Hackl, F.D. Fischer, On the relation between the principle of maximum dissipation and inelastic evolution given by dissipation potentials, PRSA, 2008](https://doi.org/10.1098/rspa.2007.0086)
 3. [C. Carstensen, Domain Decomposition for a Non-smooth Convex Minimization Problem and its Application in Plasticity, 1997](https://doi.org/10.1002/(SICI)1099-1506(199705/06)4:3<177::AID-NLA106>3.0.CO;2-B)

In [ ]:
from ngsolve import *
from ngsolve.comp import IntegrationRuleSpace
from ngsolve.fem import MinimizationCF
from netgen.geom2d import CSG2d, Circle, Rectangle
from ngsolve.webgui import Draw
SetNumThreads(1)

The problem domain is the classical plate with hole. Thanks to symmetry of the problem to be solve, we actually use only one quarter of the domain together with additional symmetry boundary conditions.
Parameters for the geometry and the material employed are taken from [4].


4. [A. Düster and E. Rank: The p-version of the fnite element method compared to an
adaptive h-version for the deformation theory of plasticity, CMAME 190 (2001) 1925-1935](https://doi.org/10.1016/S0045-7825(00)00215-2)

In [ ]:
# polynomial order for geometry and displacements
order = 3

geo = CSG2d()
circle = Circle(center=(100,100), radius=10.0, bc="curve").Maxh(1)
rect = Rectangle(pmin=(0,100), pmax=(100,200), bottom="bottom", left="left", top="top", right="right")
geo.Add(rect-circle)
mesh = Mesh(geo.GenerateMesh(maxh=5))
mesh.Curve(order)
Draw(mesh)

# points of interest for post-processing
node_A = mesh(100,200)
node_B = mesh(0,200)

We employ a $H^1$ space for the displacements and a symmetrix-matrix-valued "integration rule space" (IR space) for the internal variable $p$. For the hardening variable $\alpha$ (also internal), we simply use a scalar IR space.

In [ ]:
fes_u = VectorH1(mesh, order=order, dirichletx="right", dirichlety="bottom")
fes_ir = IntegrationRuleSpace(mesh, order=order-1)
fes_p = MatrixValued(fes_ir, symmetric=True, deviatoric=False)

# We need a coupled space as well. Note that alpha is solved for separately and thus we only have u and p here
fes = fes_u * fes_p
u, p = fes.TrialFunction()

# GridFunction for solution
gfsol = GridFunction(fes)
gfu, gfp = gfsol.components

# Save previous solution
gfsol_k = GridFunction(fes)
gfu_k, gfp_k = gfsol_k.components

#alpha_0 = 0
alpha_k = GridFunction(fes_ir)

Since the internal variables are only available on a defined set of quadrature points,
we need to use the corresponding quadrature rules for integration. This is done via

In [ ]:
# extract integration rules from IntegrationRuleSpace
irs_dx = dx(intrules=fes_ir.GetIntegrationRules())

Next we setup functions for the constitutive functions $\Psi$ and $\Phi$

In [ ]:
mu = 80193.8
kappa = 164206
Emod = (9 * kappa * mu) / (3 * kappa + mu)
nu = (3 * kappa - 2 * mu)/(2 * (3 * kappa + mu))
print("E =", Emod, ", nu =", nu)
lmbda = Emod * nu / (1 + nu)/(1 - 2 * nu)
mu = Emod / (2 * (1 + nu))
print("mu =", mu, ", lam =", lmbda)
H = 0.0
sigma_Y = sqrt(2/3)*450

def elastic_strain_energy(eps, p, mu, lam):
    return InnerProduct(2 * mu * (eps - p) + lam * Trace(eps) * Id(2), eps - p) \
            + (lam*Trace(eps) + 2 * mu * Trace(p)) * Trace(p)


def perturbed_tensor_norm(a):
    return sqrt(InnerProduct(a, a) + Trace(a)**2+1e-16)


def Psi(p, strain, p_k, alpha_k):
    strain_energy = 0.5 * elastic_strain_energy(strain, p, mu, lmbda)
    alpha = alpha_k + sigma_Y * H * perturbed_tensor_norm(p - p_k)
    hardening    = 0.5 * alpha**2
    
    return strain_energy + hardening


# Dissipation potential: For rate-independent plasticity (homogeneous of degree 1 in the rate of p) 
# this the dissipation!
Delta_t = 1 # only for formal reasons here

def Phi(p, p_k):
    return sigma_Y * perturbed_tensor_norm(p - p_k)

and loads.

In [ ]:
force = 450
# For loadsteps
loadfactor = Parameter(1)

Finally, we have all ingredients for the variational formulation: 

Find for given  $(u^k,p^k)$ a solution $(u^{k+1},p^{k+1})$ such that
\begin{align*}
\Psi(\epsilon(u^{k+1}), p^{k+1},p^{k}) + \Phi(p^{k+1},p^k)-\int f^{k+1}\cdot u^{k+1}\,dx \rightarrow \min!
\end{align*}

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += Variation( Psi(p, Sym(Grad(u)), gfp_k, alpha_k) * irs_dx ).Compile()
a += Variation( Phi(p, gfp_k) * irs_dx ).Compile()
a += Variation( -u[1] * force * loadfactor * ds("top") ).Compile()

Since Phi(p, gfp_k) is included above, we automatically obtain an "algorithmically
consistent linearization", correspondig to the local evolution

In [ ]:
evolution_objective = (Psi(p, Sym(Grad(gfu)), gfp_k, alpha_k) + Phi(p,gfp_k)).Compile()

which will be used to solve for `p` in a nested way, ie. for each "guess" for `u^{k+1}`.

Now some data structures for postprocessing...

In [ ]:
# reconstruct p for drawing
drawfes = MatrixValued(L2(mesh, order=order-1), dim=2, symmetric=True, deviatoric=True)
pd, qd = drawfes.TnT()
pdraw = GridFunction(drawfes)
ad = BilinearForm(InnerProduct(pd, qd)*irs_dx).Assemble()
invad = ad.mat.Inverse()
fd = LinearForm(InnerProduct(gfp, qd)*irs_dx)

# reconstruct alpha for drawing
drawafes = L2(mesh, order=order-1)
pda, qda = drawafes.TnT()
adraw = GridFunction(drawafes)
ada = BilinearForm(InnerProduct(pda, qda)*irs_dx).Assemble()
invada = ada.mat.Inverse()
fda = LinearForm(InnerProduct(alpha_k, qda)*irs_dx)

In order to run the problem, we need to define vectors that store respective solution data, define loadsteps
and setup a loop that iterates through the latter.

The nonlinear PDE is solved with a Newton-Raphson scheme. Thereby, in each Newton iteration
we solve a linearized PDE for the displacements and a nonlinear evolution problem at each quadratire point via *`MinimizationCF`*

In [ ]:
# a set of vectors for keeping data
vec_k = gfsol.vec.CreateVector()
w      = gfsol.vec.CreateVector()
rhs    = gfsol.vec.CreateVector()

# load steps (chosen based on experience)
loadsteps = [0.1,0.3,0.5,0.7,0.8,0.9,0.95,1.0]

# set solution to zero initially
gfsol.vec[:] = 0
alpha_k.vec[:] = 0

# evolution params (default values of MinimizationCF at the time of writing)
tol = 1e-6
maxiter = 20

# iterate through load steps
for ls in loadsteps:
    loadfactor.Set(ls)
    
    # update old solution at time t = t_k
    gfsol_k.vec.data = gfsol.vec
    with TaskManager():
        for i in range(200):
            energy_k = a.Energy(gfsol.vec)
            #print ("energy(t_k) = ", energy_k)
            
            a.AssembleLinearization(gfsol.vec)
            a.Apply(gfsol.vec, rhs)
            
            # static condensation establishes the "consistent linearization"
            w.data = a.mat.Inverse(freedofs=fes.FreeDofs(False), inverse="sparsecholesky") * rhs

            # linesearch ( with damping)
            vec_k.data = gfsol.vec
            alpha = 1
            while alpha > 1e-7:
                gfsol.vec.data -= min(1, 0.3 * (int(i / 2) + 1)) * alpha * w
                energy1 = a.Energy(gfsol.vec)
                
                # Here we solve the evolution equation as each quadrature point through evaluation of 
                # the MinimizationCF.
                gfp.Interpolate(MinimizationCF(evolution_objective, gfp, tol=tol, maxiter=maxiter))
                # MinimizationCF takes the objective function, the initial guess (most likely the previous solution),
                # tolerances and the maximum number of iterations.
                
                energy2 = a.Energy(gfsol.vec)
                
                # print ("alpha =", alpha, "energy =", energy2, " inc energy1 = ", energy1-energy_k, " inc2=", energy2-energyold)
                # input ("key")
                if energy2 < energy_k + 1e-12: 
                    break
                alpha *= 0.5
                gfsol.vec.data = vec_k
                

            err = sqrt(InnerProduct(w, rhs))
            print("step ", i, "err = ", err)
            
            # check convergence
            if err < 1e-6: break
    
    alpha_k.Interpolate(alpha_k + sigma_Y * H * sqrt(InnerProduct(gfp - gfp_k, gfp - gfp_k)))
     
    print("force = ", ls * force, ", uy_A =", gfu(node_A)[1], ", ux_B =", gfu(node_B)[0],\
          ", int u2 =", Integrate(gfu[1] * ds("top"),mesh))
    
    # for Drawing
    fd.Assemble()
    pdraw.vec.data = invad * fd.vec
    Draw(Norm(pdraw), mesh, "p")
    Draw(gfu,mesh)
    
    fda.Assemble()
    adraw.vec.data = invada * fda.vec
    #Draw(Norm(adraw), mesh, "alpha")

result: uy_A = 6.14972e-2, ux_B = 0.24662, int u2 = 22.4163

reference values [5]: (uy_A) u4y=0.24690, (ux_B) ux5 = 6.1389e-2, int u2 = 22.454

5. [R. Rannacher, F.-T. Suttmeier: A posteriori error estimation and mesh adaptation for finite element
models in elasto-plasticity, CMAME 176 (1999) 333-361](https://doi.org/10.1016/S0045-7825(98)00344-2)